In [1]:
!nvidia-smi

Sun Oct 27 09:18:38 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    30W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt

%matplotlib inline

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

print("TF Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("TF Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

TF Version:  2.0.0
Eager mode:  True
TF Hub version:  0.6.0
GPU is available


In [3]:
dataset = pd.read_csv(r'https://github.com/dipanjanS/nlp_workshop_dhs18/raw/master/Unit%2011%20-%20Sentiment%20Analysis%20-%20Unsupervised%20Learning/movie_reviews.csv.bz2', compression='bz2')
dataset['sentiment'] = [1 if record == 'positive' else 0 for record in dataset['sentiment']]
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null int64
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [4]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [5]:
reviews = dataset['review'].values
sentiments = dataset['sentiment'].values

train_reviews = reviews[:5000]
val_reviews = reviews [5000:10000]
test_reviews = reviews[10000:]



train_sentiments = sentiments[:5000]
val_sentiments = sentiments [5000:10000]
test_sentiments = sentiments[10000:]

train_reviews.shape, val_reviews.shape, test_reviews.shape

((5000,), (5000,), (40000,))

In [6]:
train_reviews[:3]

array(["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to

In [7]:
tf.compat.v1.disable_eager_execution()

In [8]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(module_url, trainable=True)

In [9]:
with tf.compat.v1.Session() as session:
    session.run([tf.compat.v1.global_variables_initializer(), 
                 tf.compat.v1.tables_initializer()])
    message_embeddings = session.run(embed(train_reviews[:3]))

message_embeddings, message_embeddings.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


(array([[-0.02637997,  0.07631321, -0.01577696, ..., -0.0692173 ,
         -0.04053345,  0.08181424],
        [ 0.04789976,  0.02754292,  0.01141358, ..., -0.00466745,
          0.02187275,  0.05876644],
        [-0.02861057,  0.02973452, -0.02516095, ..., -0.04818085,
          0.05127342,  0.10013439]], dtype=float32), (3, 512))

In [10]:
def UniversalSentenceEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

In [11]:
input_text = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
embedding = tf.keras.layers.Lambda(UniversalSentenceEmbedding, output_shape=(512, ))(input_text)
dense1 = tf.keras.layers.Dense(256, activation='relu')(embedding)
drop1 = tf.keras.layers.Dropout(0.25)(dense1)
dense2 = tf.keras.layers.Dense(256, activation='relu')(drop1)
drop2 = tf.keras.layers.Dropout(0.25)(dense2)
pred = tf.keras.layers.Dense(1, activation='sigmoid')(drop2)
model = tf.keras.models.Model(inputs=[input_text], outputs=pred)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
lambda (Lambda)              (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257   

In [12]:
EPOCHS=100
BATCH_SIZE=128

In [13]:
# Fit the model
with tf.compat.v1.Session() as session:
    session.run(tf.compat.v1.global_variables_initializer())  
    session.run(tf.compat.v1.tables_initializer())
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                      patience=3,
                                      restore_best_weights=True,
                                      verbose=1)
    
    model.fit(train_reviews, train_sentiments, 
              validation_data=(val_reviews, val_sentiments),
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              shuffle=True,
              callbacks=[es],
              verbose=1)
    
    model.save_weights('./use_model.h5')

Train on 5000 samples, validate on 5000 samples
Epoch 1/100
5000/5000 [==============================] - 22s 4ms/sample - loss: 0.4879 - accuracy: 0.7762 - val_loss: 0.3804 - val_accuracy: 0.8326
Epoch 2/100
5000/5000 [==============================] - 20s 4ms/sample - loss: 0.3610 - accuracy: 0.8356 - val_loss: 0.3692 - val_accuracy: 0.8358
Epoch 3/100
5000/5000 [==============================] - 20s 4ms/sample - loss: 0.3443 - accuracy: 0.8474 - val_loss: 0.3643 - val_accuracy: 0.8394
Epoch 4/100
5000/5000 [==============================] - 20s 4ms/sample - loss: 0.3348 - accuracy: 0.8538 - val_loss: 0.3643 - val_accuracy: 0.8400
Epoch 5/100
5000/5000 [==============================] - 20s 4ms/sample - loss: 0.3260 - accuracy: 0.8534 - val_loss: 0.3645 - val_accuracy: 0.8394
Epoch 6/100
5000/5000 [==============================] - 20s 4ms/sample - loss: 0.3209 - accuracy: 0.8534 - val_loss: 0.3593 - val_accuracy: 0.8398
Epoch 7/100
5000/5000 [==============================] - 20s 4ms

In [14]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(module_url, trainable=False)

def UniversalSentenceEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

input_text = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
embedding = tf.keras.layers.Lambda(UniversalSentenceEmbedding, output_shape=(512, ))(input_text)
dense1 = tf.keras.layers.Dense(256, activation='relu')(embedding)
drop1 = tf.keras.layers.Dropout(0.25)(dense1)
dense2 = tf.keras.layers.Dense(256, activation='relu')(drop1)
drop2 = tf.keras.layers.Dropout(0.25)(dense2)
pred = tf.keras.layers.Dense(1, activation='sigmoid')(drop2)
model = tf.keras.models.Model(inputs=[input_text], outputs=pred)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [15]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Final evaluation of the model
with tf.compat.v1.Session() as session:
    session.run(tf.compat.v1.global_variables_initializer())
    session.run(tf.compat.v1.tables_initializer())
    
    model.load_weights('./use_model.h5')
    
    predictions = [1 if pr > 0.5 else 0 
                   for pr in model.predict(test_reviews, batch_size=512, verbose=0).ravel()]

print("Accuracy: %.2f%%" % (accuracy_score(test_sentiments, predictions)*100))
print(classification_report(test_sentiments, predictions))
pd.DataFrame(confusion_matrix(test_sentiments, predictions))

Accuracy: 83.69%
             precision    recall  f1-score   support

          0       0.84      0.84      0.84     20028
          1       0.84      0.84      0.84     19972

avg / total       0.84      0.84      0.84     40000



,0,1
0,16750,3278
1,3248,16724
